In [2]:
import pickle
import numpy as np
from __future__ import absolute_import, division, print_function
import pandas as pd
import os
import nltk
from collections import defaultdict
from os import listdir
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
# load the data into panda dataframe
data_file_name = "dataset/reviews_Musical_Instruments.json"
#raw_df = pd.read_json(data_file_name, lines=True)
print("Data loaded")

Data loaded


In [3]:
with open('commentdata.pkl', 'rb') as handle:
    list_of_comments = pickle.load(handle)

In [3]:
import multiprocessing

#Dimensionality of the resulting word vectors
num_features = 500

#Minimum word count threshold
min_word_count = 3

#Number of threads to run in parallel
num_workers = multiprocessing.cpu_count()

#Context window length
context_size = 4

#Seed for the RNG, to make the result reproducible
seed = 1

In [19]:
mypath='train/pos'
mypath1='train/neg'
documents = [f for f in listdir(mypath) if f.endswith('.txt')]
documents1 = [f for f in listdir(mypath1) if f.endswith('.txt')]

In [ ]:
mypath='unsup/'
def getMovieDat():
   
   #documents = [f for f in listdir(mypath) if f.startswith('data_')]
   #documents = [os.listdir(mypath)]
   documents = [f for f in listdir(mypath) if f.endswith('.txt')]
   lmtzr = WordNetLemmatizer()
   docs=[]
   sent_corp=defaultdict(list)
   tokenizer = RegexpTokenizer(r'\w+')
   wrd_freq=Counter()
   for doc in tqdm(documents):
       with open(mypath+doc,'r', errors = 'ignore') as fp:
           lines = ' '.join(fp.readlines())
           sent = sent_tokenize(lines)
           tmp = ''
           for sen in sent:
               tokens=tokenizer.tokenize(sen)
               tagged = nltk.pos_tag(tokens)
               nouns = [word for word,pos in tagged \
                        if (pos == 'NN' or pos == 'NNP' or pos == 'NNS'\
                            or pos == 'NNPS' or pos=='JJ')]
               downcased = [x.lower() for x in nouns]
               downcased = [lmtzr.lemmatize(x) for x in downcased]
               downcased = [x for x in downcased if x not in stopwords.words('english')]
               if len(downcased)>1:
                   sent_corp[doc].append([set(downcased)]+[sen])
                   tmp += ' '.join(downcased)
                   for x in downcased: wrd_freq[x]+=1
           docs.append(tmp)
   tp_wrds=sorted(wrd_freq.items(), key=itemgetter(1), reverse=True)
   return (docs,tp_wrds,sent_corp)
docs,tp_wrds,sent_corp = getMovieDat()

In [24]:
docs=[]
tokenizer = RegexpTokenizer(r'\w+')
for doc in tqdm(documents):
    with open(mypath+doc,'r', errors = 'ignore') as fp:
        lines = ' '.join(fp.readlines())
        sent = sent_tokenize(lines)
        tmp = ''
        for sen in sent:
            tokens=tokenizer.tokenize(sen)
            downcased = [x.lower() for x in tokens]
            tmp.append(downcased)
        docs.append(tmp)

  0%|          | 0/12500 [00:00<?, ?it/s]


TypeError: 'errors' is an invalid keyword argument for this function

In [7]:
final_list = []
for i in range(len(docs)):
    appended_list = [] 
    for video in docs[i]:
        for comment in video:
            appended_list.append(comment)
    final_list.append(appended_list)

In [8]:
import gensim

doc2vec_model = gensim.models.doc2vec.Doc2Vec(
    seed=seed,
    workers=num_workers, 
    size=num_features, 
    min_count=min_word_count, 
    window=context_size)

/home/dmmlasu/.local/lib/python2.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [9]:
documents = [gensim.models.doc2vec.TaggedDocument(s,[i]) for i,s in enumerate(final_list)]

In [10]:
len(documents)

2238

In [11]:
doc2vec_model.build_vocab(documents)
print("The vocabword2vec_model.iterulary is built")
print("Word2Vec vocabulary length: ", len(doc2vec_model.wv.vocab))

The vocabword2vec_model.iterulary is built
Word2Vec vocabulary length:  82338


In [12]:
#Start training the model
doc2vec_model.train(documents=documents,total_examples=doc2vec_model.corpus_count,epochs=doc2vec_model.iter)
print("Training finished")

/home/dmmlasu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Training finished


In [13]:
#Save the model
doc2vec_model.save("youtube_data1.d2v")
print("Model saved")

Model saved
